In [7]:
import datasets 
import transformers
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F


from nmt_clean.config import config
from nmt_clean.load_data import load_training_data, load_validation_data
from nmt_clean.metrics import compute_sacreBLEU as compute_metrics
from nmt_clean.preprocess import Many2OneProcessor


from transformers import EarlyStoppingCallback
from transformers import AutoModelForQuestionAnswering


### Quantization not working !!! BLEU scores are not straightforward to optimize for in optimum/neural compressor
# The directory where the quantized model will be saved
save_dir = "test-quant"

config['data_dir'] = f'/home/ali/Documents/repos/datasets/salt/v7-dataset/' #FIXME use os.path.join
config['model_checkpoint'] = "/home/ali/Documents/repos/nmt_checkpoints/mul_en_kaggle_hf_1-2/output-mul-en/checkpoint-400"
tokenizer = transformers.AutoTokenizer.from_pretrained(config["model_checkpoint"])
main_model = transformers.AutoModelForSeq2SeqLM.from_pretrained(config['model_checkpoint'])
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer, model = main_model) 

metric = datasets.load_metric('sacrebleu')

processor = Many2OneProcessor()



In [11]:
main_model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(64172, 512, padding_idx=64171)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(64172, 512, padding_idx=64171)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1

In [36]:


import torch
from torchvision.models import resnet18
import torch_pruning as tp

# Importance criteria
intexample_inputs = torch.randn(1, 128, 64172)
imp = tp.importance.MagnitudeImportance(p=2)

ignored_layers = []
for m in main_model.modules():
    try:
        if  m.out_features == 64172:
            print("toz")
            ignored_layers.append(m) # DO NOT prune the final classifier!
        if  m.in_features == 64172:
            print("toz")
            ignored_layers.append(m) # DO NOT prune the final classifier!
        if isinstance(m, torch.nn.Embedding):
            print("toz")
            ignored_layers.append(m) # DO NOT prune the final classifier!
            
    except: 
        pass


toz


In [37]:
iterative_steps = 5 # progressive pruning
pruner = tp.pruner.MagnitudePruner(
    main_model,
    example_inputs,
    importance=imp,
    iterative_steps=iterative_steps,
    ch_sparsity=0.5, # remove 50% channels, ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
    ignored_layers=ignored_layers,
)

base_macs, base_nparams = tp.utils.count_ops_and_params(model, example_inputs)
for i in range(iterative_steps):
    pruner.step()
    macs, nparams = tp.utils.count_ops_and_params(model, example_inputs)
    # finetune your model here
    # finetune(model)
    # ...

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [38]:
result[-2]

LayerNorm((512,), eps=1e-05, elementwise_affine=True)